# Yield Curve Model Testing Notebook

This notebook exercises the `YieldCurve` class by:
1. Building a flat dummy curve for SOFR and USD LIBOR.
2. Computing spot discount factors for various tenors.
3. Computing forward rates for overnight and IBOR indices.

---

## 1) Import dependencies

We start by importing the necessary libraries and the `YieldCurve` class.

In [1]:
import pandas as pd
import numpy as np
from fixedincomelib.yield_curve.yield_curve_model import YieldCurve
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection


## 2) Define dummy curve data

Create a small DataFrame with dummy rates for two indices, at 1M, 3M, and 6M tenors.

In [2]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-05-05 x 2025-08-05", 97.25],
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-08-05 x 2025-11-05", 97.40],
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-11-05 x 2026-02-05", 97.55],

        ["RFR SWAP", "USD-SOFR-OIS", "1Y",  0.0450],
        ["RFR SWAP", "USD-SOFR-OIS", "2Y",  0.0425],
        ["RFR SWAP", "USD-SOFR-OIS", "5Y",  0.0370],
        ["RFR SWAP", "USD-SOFR-OIS", "10Y", 0.0350],
    ],
    columns=["DATA TYPE", "DATA CONVENTION", "AXIS", "VALUE"],
)

data_objs, dc = build_yc_data_collection(MARKET_DF)


## 3) Define build methods

Specify how each index should be interpolated; here we use piecewise constant interpolation.

In [3]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
}]


## 4) Instantiate the yield curve

Set the valuation date and build the `YieldCurve` object.

In [4]:
value_date = "2025-05-05"
yc = YieldCurve(valueDate=value_date, dataCollection=dc, buildMethodCollection=build_methods)

comp = yc.retrieveComponent("SOFR-1B")
print("Calibration summary:", getattr(comp, "_calibration_summary", {}))

['SOFR-1B']
Calibration summary: {}


In [5]:
yc.components.keys()

dict_keys(['SOFR-1B'])

## 5) Compute and display spot discount factors

We pick several (index, date) pairs to show discount factors.

In [6]:
print("--- Spot Discount Factors ---")
tests = [
    ("SOFR-1B",          "2025-07-26"),
    ("SOFR-1B",          "2025-09-26"),
]
for idx, dt in tests:
    df_val = yc.discountFactor(idx, dt)
    print(f"{idx:>18} @ {dt}: DF = {df_val:.6f}")


--- Spot Discount Factors ---
           SOFR-1B @ 2025-07-26: DF = 0.993717
           SOFR-1B @ 2025-09-26: DF = 0.989302


## 6) Compute and display forward rates

Finally, we compute 1M and 3M overnight forwards, and a 3M LIBOR forward:

In [7]:
print("\n--- Forward Rates ---")
fwd_1m    = yc.forward("SOFR-1B",          value_date, "1M")
fwd_3m    = yc.forward("SOFR-1B",          value_date, "3M")

print(f"SOFR 1M forward:  {fwd_1m:.6%}")
print(f"SOFR 3M forward:  {fwd_3m:.6%}")



--- Forward Rates ---
SOFR 1M forward:  2.714082%
SOFR 3M forward:  2.750000%


In [8]:
starting_date = "2026-09-12"
tenor = "3M"

yc.forward("SOFR-1B", starting_date, tenor)

np.float64(0.039429259824423406)